In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
#cuda.use('gpu0')

Using Theano backend.


## Read data

In [4]:
#path = "data/movielens/big/"
path = "data/movielens/small/"
batch_size=64

In [13]:
ratings = pd.read_csv(path+'ratings.csv')
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])
users = ratings.userId.unique()
movies = ratings.movieId.unique()
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [14]:
val

,userId,movieId,rating,timestamp
6,1,1287,2.0,1260759187
11,1,1405,1.0,1260759203
12,1,1953,4.0,1260759191
18,1,2968,1.0,1260759200
25,2,52,3.0,835356031
30,2,153,4.0,835355441
31,2,161,3.0,835355493
34,2,185,3.0,835355511
35,2,186,3.0,835355664
43,2,261,4.0,835355681


## Create test/validation set

In [15]:
n_factors = 50
np.random.seed = 42

In [16]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [17]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [18]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [21]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')
nn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________________________

In [22]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80117 samples, validate on 19887 samples
Epoch 1/8
80117/80117 [==============================] - 8s - loss: 1.2303 - val_loss: 0.8006
Epoch 2/8
80117/80117 [==============================] - 7s - loss: 0.8988 - val_loss: 0.7973
Epoch 3/8
80117/80117 [==============================] - 7s - loss: 0.8673 - val_loss: 0.7860
Epoch 4/8
80117/80117 [==============================] - 7s - loss: 0.8529 - val_loss: 0.7849
Epoch 5/8
80117/80117 [==============================] - 7s - loss: 0.8370 - val_loss: 0.7916
Epoch 6/8
80117/80117 [==============================] - 7s - loss: 0.8225 - val_loss: 0.7932
Epoch 7/8
80117/80117 [==============================] - 7s - loss: 0.8112 - val_loss: 0.7946
Epoch 8/8
80117/80117 [==============================] - 7s - loss: 0.7997 - val_loss: 0.7978


0            0
2            2
3            3
4            4
5            5
6            6
7            7
8            8
9            9
10          10
11          11
13          13
14          14
15          15
16          16
17          17
19          19
20          20
22          22
23          23
24          24
26          26
27          27
28          28
29          29
30          30
32          32
34          34
36          36
37          37
          ... 
99971      352
99972     1126
99973     1129
99974      473
99975      354
99976      355
99977     5577
99978      477
99979      478
99981      479
99982      480
99983      359
99984     1225
99985     1240
99986      361
99987      126
99988     1260
99989      483
99990      362
99991      127
99992      364
99993     1299
99995      486
99996     1308
99997      365
99998     2930
99999     7005
100000    4771
100001    1329
100002    1331
Name: movieId, dtype: int64